# iNaturalist status updates by state

Using the file produced in the collate-status-taxa.ipynb: `inat-aust-status-taxa.csv`, generate lists to update iNaturalist statuses

**Next steps:**
State by state establish the changes that need to be made:
    a. new - any new species that appear in the state lists but do not have a status in inaturalist (new template)
    b. updates - any changes to information which was added by us previously (user_id = 708886) (update template, action='UPDATE')
    c. removals - any statuses which were added by us previously (user_id = 708886) list which are incorrect (update template, action='REMOVE')
    d. flags - are there any statuses by other users that need to be flagged?

## Prep - common to all states
1. Read in the inaturalist statuses & filter out this state
2. Read in the inaturalist taxa list
3. Read in the state sensitive list
4. Attempt to match the state statuses to an IUCN equivalent


### 1. iNaturalist statuses

In [50]:
import pandas as pd
import sys
import os
projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/" # basedir for this gh project
sys.path.append(os.path.abspath(projectdir + "source-code/includes"))
import list_functions as lf

sourcedir = projectdir + "source-data/inaturalist-statuses/"
listdir = projectdir + "current-lists/"


# read in the statuses
taxastatus = pd.read_csv(sourcedir + "inat-aust-status-taxa.csv", encoding='UTF-8',na_filter=False,dtype=str) ## Read inaturalist conservation statuses file
taxastatus.head(3)

,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,166449,38493,1138587,7830,,Flora and Fauna Guarantee Act 1988,CR,,,obscured,...,Eulamprus,kosciuskoi,,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,,,
1,234788,918383,702203,9994,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,
2,234789,918383,702203,7308,,Atlas of Living Australia,LC,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,


In [51]:
def filter_state_statuses(stateregex: str, urlregex: str):
    authoritydf = taxastatus['authority'].drop_duplicates().sort_values()
    authoritydf = authoritydf[pd.Series(authoritydf).str.contains(stateregex)]
    urldf = taxastatus['url'].drop_duplicates().sort_values()
    urldf = urldf[pd.Series(urldf).str.contains(urlregex)]
    placedisplaydf = taxastatus['place_display_name'].drop_duplicates().sort_values()
    placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains(stateregex)]
    placedf = taxastatus['place_name'].drop_duplicates().sort_values()
    placedf = placedf[pd.Series(placedf).str.contains(stateregex)]
    # concat all and remove duplicates
    statedf = pd.concat([taxastatus.apply(lambda row: row[taxastatus['place_display_name'].isin(placedisplaydf)]),
                         taxastatus.apply(lambda row: row[taxastatus['place_name'].isin(placedf)]),
                         taxastatus.apply(lambda row: row[taxastatus['url'].isin(urldf)]),
                         taxastatus.apply(
                             lambda row: row[taxastatus['authority'].isin(authoritydf)])]).drop_duplicates()
    return statedf.sort_values(['taxon_id', 'user_id'])
inatstatuses = filter_state_statuses("ACT Government", ".act.gov")
inatstatuses.rename(columns={'id':'status_id','id_y':'taxon_id_y'},inplace=True)
inatstatuses

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
2465,152234,100611,,12986,16649,ACT Government,vulnerable,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Euastacus,armatus,,2022-06-06T16:36:21Z,Euastacus armatus,species,http://www.iucnredlist.org/apps/redlist/details,,,
270,180956,1130138,3669610,12986,,ACT Government,EN,http://www.environment.gov.au/cgi-bin/sprat/pu...,,obscured,...,Prasophyllum,petilum,,2022-08-18T08:30:22Z,Prasophyllum petilum,species,http://www.catalogueoflife.org/annual-checklis...,,,
3305,266735,1255190,5159763,12986,,ACT Government,Endangered,https://www.environment.act.gov.au/nature-cons...,Listed as endangered in the ACT,obscured,...,Lepidium,ginninderrense,,2021-07-27T21:58:53Z,Lepidium ginninderrense,species,http://www.catalogueoflife.org/annual-checklis...,,,
2662,152233,138646,708886,12986,16649,ACT Government,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Swainsona,recta,,2022-06-08T16:59:25Z,Swainsona recta,species,,,,
1078,152222,25262,708886,12986,16649,ACT Government,endangered,https://lists.ala.org.au/speciesListItem/list/...,,private,...,Pseudophryne,pengilleyi,,2018-11-27T04:54:16Z,Pseudophryne pengilleyi,species,http://research.amnh.org/vz/herpetology/amphib...,,,
1082,152227,36937,708886,12986,16649,ACT Government,vulnerable,https://lists.ala.org.au/speciesListItem/list/...,,private,...,Delma,impar,,2019-06-06T05:58:40Z,Delma impar,species,http://www.iucnredlist.org/apps/redlist/detail...,,,
1090,152221,36957,708886,12986,16649,ACT Government,vulnerable,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Aprasia,parapulchella,,2022-04-07T23:19:10Z,Aprasia parapulchella,species,http://reptile-database.reptarium.cz/search.ph...,,,
1076,152230,39441,708886,12986,16649,ACT Government,vulnerable,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Varanus,rosenbergi,,2021-07-28T09:18:05Z,Varanus rosenbergi,species,http://www.iucnredlist.org/apps/redlist/detail...,,,
1079,152224,45210,708886,12986,16649,ACT Government,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Pseudomys,fumeus,,2019-11-22T22:45:54Z,Pseudomys fumeus,species,http://www.catalogueoflife.org/annual-checklis...,,,
1088,152235,51333,708886,12986,16649,ACT Government,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Macquaria,australasica,,2019-11-24T03:18:43Z,Macquaria australasica,species,http://www.fishbase.org,,,


### 2. iNaturalist taxonomy

In [52]:
# Output files contain these fields
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# so we need to match species from the state lists to the inat taxa to get the taxon_id

import zipfile
url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

z=zipfile.ZipFile(sourcedir + filename)

with z.open('taxa.csv') as from_archive:
    inattaxa = pd.read_csv(from_archive,dtype=str)
z.close()
inattaxa.head(3)


,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
0,1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/48460,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-02T06:05:44Z,Animalia,kingdom,http://www.catalogueoflife.org/annual-checklis...
1,2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/1,Animalia,Chordata,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-23T00:40:18Z,Chordata,phylum,http://www.catalogueoflife.org/annual-checklis...
2,3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/355675,Animalia,Chordata,Aves,NaN,NaN,NaN,NaN,NaN,2022-12-27T07:33:16Z,Aves,class,http://www.catalogueoflife.org/annual-checklis...


### 3. State lists
Sensitive list: `geoprivacy` = `obscured`
overrides

In [53]:
%%script echo skipping # comment this line to download dataset from lists.ala.org.au the web and save locally
# Download lists data. Retrieve binomial and trinomial names from GBIF. Save locally to CSV

sensitivelist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr2627?max=10000&includeKVP=true")
sensitivelist = lf.kvp_to_columns(sensitivelist)
sensitivelist.to_csv(sourcedir + "act-ala-sensitive.csv", index=False)

conservationlist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr649?max=10000&includeKVP=true")
conservationlist = lf.kvp_to_columns(conservationlist)
conservationlist.to_csv(sourcedir + "act-ala-conservation.csv", index=False)


Couldn't find program: 'echo'


In [54]:
# Read sensitive list data
sensitivelist = pd.read_csv(sourcedir + "act-ala-sensitive.csv", dtype=str)
sensitivelist = sensitivelist.rename(columns={'T S Profile I D':'bionetTaxonID'})
sensitivelist = sensitivelist.rename(columns={'conservation status': 'status'})
sensitivelist['bionet_geoprivacy'] = 'obscured'
sensitivelist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,generalisation description,vernacular name,status,Authority,category,generalisation,Reason,bionet_geoprivacy
0,510630,Prasophyllum petilum,Tarengo Leek Orchid,Prasophyllum petilum,https://id.biodiversity.org.au/taxon/apni/5140...,dr2627,"[{'key': 'generalisation description', 'value'...",Generalise to 0.01 or 1km,A leek orchid,Endangered,ACT Nature Conservation Act 2014,High,1km,Disturbance and collection,obscured
1,510637,Corunastylis ectopa,NaN,Genoplesium ectopum,https://id.biodiversity.org.au/taxon/apni/5140...,dr2627,"[{'key': 'generalisation description', 'value'...",Generalise to 0.01 or 1km,Brindabella Midge Orchid,Endangered,ACT Nature Conservation Act 2014,High,1km,Disturbance and collection,obscured
2,510639,Caladenia actensis,Canberra Spider Orchid,Caladenia actensis,https://id.biodiversity.org.au/taxon/apni/5139...,dr2627,"[{'key': 'generalisation description', 'value'...",Generalise to 0.01 or 1km,Canberra Spider Orchid,Endangered,ACT Nature Conservation Act 2014,High,1km,Disturbance and collection,obscured
3,510635,Lepidium ginninderrense,Ginninderra Peppercress,Lepidium ginninderrense,https://id.biodiversity.org.au/taxon/apni/5126...,dr2627,"[{'key': 'generalisation description', 'value'...",Generalise to 0.01 or 1km,Ginninderra Peppercress,Endangered,ACT Nature Conservation Act 2014,High,1km,Disturbance and collection,obscured
4,510646,Pterostylis oreophila,Blue-tongued Orchid,Pterostylis oreophila,https://id.biodiversity.org.au/taxon/apni/5141...,dr2627,"[{'key': 'generalisation description', 'value'...",Generalise to 0.01 or 1km,Kiandra Greenhood,Critically endangered,Environment Protection and Biodiversity Conser...,High,1km,Disturbance and collection,obscured
5,510644,Swainsona recta,Small Purple-pea,Swainsona recta,https://id.biodiversity.org.au/node/apni/2916272,dr2627,"[{'key': 'generalisation description', 'value'...",Generalise to 0.01 or 1km,Small Purple Pea,Endangered,ACT Nature Conservation Act 2014,High,1km,Disturbance and collection,obscured
6,510634,Muehlenbeckia tuggeranong,Tuggeranong Lignum,Muehlenbeckia tuggeranong,https://id.biodiversity.org.au/node/apni/2896218,dr2627,"[{'key': 'generalisation description', 'value'...",Generalise to 0.01 or 1km,Tuggeranong Lignum,Endangered,ACT Nature Conservation Act 2014,High,1km,Disturbance and collection,obscured
7,510638,Tympanocryptis pinguicolla,Grassland Earless Dragon,Tympanocryptis pinguicolla,https://biodiversity.org.au/afd/taxa/5bceebc1-...,dr2627,"[{'key': 'generalisation description', 'value'...",Generalise to 0.01 or 1km,NaN,Endangered,ACT Nature Conservation Act 2014,Very High,1km,Disturbance and collection,obscured
8,510627,Pseudophryne pengilleyi,Northern Corroboree Frog,Pseudophryne pengilleyi,https://biodiversity.org.au/afd/taxa/cc5315bf-...,dr2627,"[{'key': 'generalisation description', 'value'...",Generalise to 0.01 or 1km,NaN,Critically endangered,ACT Nature Conservation Act 2014,Very High,1km,Disturbance and collection,obscured
9,510640,Perunga ochracea,Perunga Grasshopper,Perunga ochracea,https://biodiversity.org.au/afd/taxa/ab03e158-...,dr2627,"[{'key': 'generalisation description', 'value'...",Generalise to 0.01 or 1km,Perunga Grasshopper,Vulnerable,ACT Nature Conservation Act 2014,High,1km,Disturbance and collection,obscured


In [55]:
# Read conservation list data
conservationlist = pd.read_csv(sourcedir + "act-ala-conservation.csv", dtype=str)
conservationlist = conservationlist.rename(columns={'T S Profile I D':'bionetTaxonID'})
conservationlist['bionet_geoprivacy'] = 'open'
conservationlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,vernacular name,status,sourceStatus,taxonRemarks,bionet_geoprivacy
0,4026202,Anthochaera phrygia,Regent Honeyeater,Anthochaera (Xanthomyza) phrygia,https://biodiversity.org.au/afd/taxa/31869a0e-...,dr649,"[{'key': 'vernacular name', 'value': 'Regent H...",Regent Honeyeater,Critically Endangered,Critically Endangered,NaN,open
1,4026236,Lathamus discolor,Swift Parrot,Lathamus discolor,https://biodiversity.org.au/afd/taxa/4cb195fd-...,dr649,"[{'key': 'vernacular name', 'value': 'Swift Pa...",Swift Parrot,Critically Endangered,Critically Endangered,NaN,open
2,4026233,Pseudophryne pengilleyi,Northern Corroboree Frog,Pseudophryne pengilleyi,https://biodiversity.org.au/afd/taxa/cc5315bf-...,dr649,"[{'key': 'vernacular name', 'value': 'Northern...",Northern Corroboree Frog,Critically Endangered,Critically Endangered,NaN,open
3,4026223,Caladenia actensis,Canberra Spider Orchid,Caladenia actensis,https://id.biodiversity.org.au/taxon/apni/5139...,dr649,"[{'key': 'vernacular name', 'value': 'Canberra...",Canberra Spider Orchid,Critically Endangered,Critically Endangered,NaN,open
4,4026207,Corunastylis ectopa,NaN,Genoplesium ectopum,https://id.biodiversity.org.au/taxon/apni/5140...,dr649,"[{'key': 'vernacular name', 'value': 'Brindabe...",Brindabella Midge Orchid,Critically Endangered,Critically Endangered,NaN,open
5,4026250,Pterostylis oreophila,Blue-tongued Orchid,Pterostylis oreophila,https://id.biodiversity.org.au/taxon/apni/5141...,dr649,"[{'key': 'vernacular name', 'value': 'Kiandra ...",Kiandra Greenhood,Critically Endangered,Critically Endangered,NaN,open
6,4026199,Litoria castanea,Yellow-spotted Bell Frog,Litoria castanea,https://biodiversity.org.au/afd/taxa/898034f7-...,dr649,"[{'key': 'vernacular name', 'value': 'Yellow-s...",Yellow-spotted Bell Frog,Critically Endangered,Critically Endangered,locally extinct,open
7,4026217,Gentiana baeuerlenii,Baeuerlen's Gentian,Gentiana baeuerlenii,https://id.biodiversity.org.au/node/apni/2892776,dr649,"[{'key': 'vernacular name', 'value': ""Baeuerle...",Baeuerlen's Gentian,Endangered,Endangered,NaN,open
8,4026206,Prasophyllum petilum,Tarengo Leek Orchid,Prasophyllum petilum,https://id.biodiversity.org.au/taxon/apni/5140...,dr649,"[{'key': 'vernacular name', 'value': 'Tarengo ...",Tarengo Leek Orchid,Endangered,Endangered,NaN,open
9,4026230,Rutidosis leptorhynchoides,Button Wrinklewort,Rutidosis leptorhynchoides,https://id.biodiversity.org.au/node/apni/7062437,dr649,"[{'key': 'vernacular name', 'value': 'Button W...",Button Wrinklewort,Endangered,Endangered,NaN,open


In [56]:
# join them in a way that works for inat (eg sensitive list, geoprivacy = 'obscured'
statelist = pd.concat([sensitivelist[['name', 'status', 'bionet_geoprivacy', 'lsid']],
                    conservationlist[['name', 'status', 'bionet_geoprivacy', 'lsid']]]).drop_duplicates()
# retrieve binomial and trinomial names from GBIF
parsednames = lf.gbifparse(statelist)
parsednames.to_csv(sourcedir + "act-gbif.csv", index=False)
statelist = statelist.merge(parsednames[['scientificName','canonicalName','canonicalNameComplete','type','rankMarker']],how="left",left_on="name",right_on="scientificName")
numfullstatelist = len(statelist.index)
statelist['scientificName'] = statelist['canonicalName']
statelist

,name,status,bionet_geoprivacy,lsid,scientificName,canonicalName,canonicalNameComplete,type,rankMarker
0,Prasophyllum petilum,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,Prasophyllum petilum,Prasophyllum petilum,Prasophyllum petilum,SCIENTIFIC,sp.
1,Prasophyllum petilum,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,Prasophyllum petilum,Prasophyllum petilum,Prasophyllum petilum,SCIENTIFIC,sp.
2,Corunastylis ectopa,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,Corunastylis ectopa,Corunastylis ectopa,Corunastylis ectopa,SCIENTIFIC,sp.
3,Corunastylis ectopa,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,Corunastylis ectopa,Corunastylis ectopa,Corunastylis ectopa,SCIENTIFIC,sp.
4,Caladenia actensis,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5139...,Caladenia actensis,Caladenia actensis,Caladenia actensis,SCIENTIFIC,sp.
...,...,...,...,...,...,...,...,...,...
109,Eucalyptus aggregata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2900093,Eucalyptus aggregata,Eucalyptus aggregata,Eucalyptus aggregata,SCIENTIFIC,sp.
110,Pomaderris pallida,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2897349,Pomaderris pallida,Pomaderris pallida,Pomaderris pallida,SCIENTIFIC,sp.
111,Thesium australe,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2891975,Thesium australe,Thesium australe,Thesium australe,SCIENTIFIC,sp.
112,Hirundapus caudacutus,Vulnerable,open,https://biodiversity.org.au/afd/taxa/6485cd0c-...,Hirundapus caudacutus,Hirundapus caudacutus,Hirundapus caudacutus,SCIENTIFIC,sp.


In [57]:

# Identify records that won't comply with iNaturalist species names
noncomply = statelist[statelist['type'].isin(['INFORMAL','CULTIVAR','HYBRID']) ]
noncomply

,name,status,bionet_geoprivacy,lsid,scientificName,canonicalName,canonicalNameComplete,type,rankMarker


In [58]:
# remove records that do not comply
statelist = statelist[~statelist['type'].isin(['INFORMAL','CULTIVAR','HYBRID']) ]
statelist = pd.DataFrame(statelist[['scientificName','status','bionet_geoprivacy','lsid']])
statelist

,scientificName,status,bionet_geoprivacy,lsid
0,Prasophyllum petilum,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...
1,Prasophyllum petilum,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...
2,Corunastylis ectopa,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...
3,Corunastylis ectopa,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...
4,Caladenia actensis,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5139...
...,...,...,...,...
109,Eucalyptus aggregata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2900093
110,Pomaderris pallida,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2897349
111,Thesium australe,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2891975
112,Hirundapus caudacutus,Vulnerable,open,https://biodiversity.org.au/afd/taxa/6485cd0c-...


In [59]:
parsednames['type'].unique()

array(['SCIENTIFIC'], dtype=object)

In [60]:
# check for duplicates with conflicting information
dupinformation = statelist.groupby('scientificName').filter(lambda x: len(x) > 1)#.sort('size',ascending=False)
dupinformation

,scientificName,status,bionet_geoprivacy,lsid
0,Prasophyllum petilum,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...
1,Prasophyllum petilum,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...
2,Corunastylis ectopa,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...
3,Corunastylis ectopa,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...
4,Caladenia actensis,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5139...
...,...,...,...,...
86,Euastacus armatus,Vulnerable,open,https://biodiversity.org.au/afd/taxa/98156289-...
87,Perunga ochracea,Vulnerable,open,https://biodiversity.org.au/afd/taxa/ab03e158-...
88,Perunga ochracea,Vulnerable,open,https://biodiversity.org.au/afd/taxa/ab03e158-...
97,Aprasia parapulchella,Vulnerable,open,https://biodiversity.org.au/afd/taxa/0d74fa05-...


### 4. Equivalent IUCN statuses

In [61]:
# iucn_statuses = {'Not Evaluated', 'Data Deficient', 'Least Concern', 'Near Threatened', 'Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct in the Wild'. 'Extinct'}
statelist.groupby(['status'])['status'].count()

status
Conservation Dependent     1
Critically Endangered     11
Critically endangered      4
Endangered                53
Rare                       3
Vulnerable                41
Name: status, dtype: int64

In [62]:
iucnStatusMappings = {
    'Critically endangered':'Critically Endangered',
    'Rare':'Vulnerable',
    'Conservation Dependent':'Vulnerable'
}

### 5. Determine best place ID to use

In [63]:
inatstatuses.groupby(['place_id','place_name','place_display_name'])['place_id'].count()
# looks like 6825

place_id  place_name                    place_display_name              
12986     Australian Capital Territory  Australian Capital Territory, AU    18
Name: place_id, dtype: int64

## Merge iNaturalist statuses with State sensitive list on scientificName

1. Match - updates, even if the statuses are the same we'll update the links and values anyway
2. No match - statuses to be added (additions)
   1.1 No match and no taxnomy - search for synonyms
   1.2 No match
3. Merge the other direction to see if there are deletes?


In [64]:
# join to see which lists already have a status in inaturalist based on scientificName
mergedstatuses = statelist[['scientificName','status','bionet_geoprivacy', 'lsid']].merge(inatstatuses[['status_id','scientificName','taxon_id','user_id','description','iucn','authority','status','geoprivacy','place_id','place_display_name']],how="left",left_on='scientificName',right_on='scientificName',suffixes=(None,'_inat')).sort_values(['scientificName'])
mergedstatuses


,scientificName,status,bionet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,status_inat,geoprivacy,place_id,place_display_name
43,Anthochaera phrygia,Critically Endangered,open,https://biodiversity.org.au/afd/taxa/31869a0e-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,Aprasia parapulchella,Vulnerable,open,https://biodiversity.org.au/afd/taxa/0d74fa05-...,152221,36957,708886,,30,ACT Government,vulnerable,obscured,12986,"Australian Capital Territory, AU"
98,Aprasia parapulchella,Vulnerable,open,https://biodiversity.org.au/afd/taxa/0d74fa05-...,152221,36957,708886,,30,ACT Government,vulnerable,obscured,12986,"Australian Capital Territory, AU"
22,Aprasia parapulchella,Vulnerable,obscured,https://biodiversity.org.au/afd/taxa/0d74fa05-...,152221,36957,708886,,30,ACT Government,vulnerable,obscured,12986,"Australian Capital Territory, AU"
21,Aprasia parapulchella,Vulnerable,obscured,https://biodiversity.org.au/afd/taxa/0d74fa05-...,152221,36957,708886,,30,ACT Government,vulnerable,obscured,12986,"Australian Capital Territory, AU"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,Tympanocryptis pinguicolla,Endangered,obscured,https://biodiversity.org.au/afd/taxa/5bceebc1-...,152228,73985,708886,,40,ACT Government,endangered,private,12986,"Australian Capital Territory, AU"
14,Tympanocryptis pinguicolla,Endangered,obscured,https://biodiversity.org.au/afd/taxa/5bceebc1-...,152228,73985,708886,,40,ACT Government,endangered,private,12986,"Australian Capital Territory, AU"
63,Tympanocryptis pinguicolla,Endangered,open,https://biodiversity.org.au/afd/taxa/5bceebc1-...,152228,73985,708886,,40,ACT Government,endangered,private,12986,"Australian Capital Territory, AU"
62,Tympanocryptis pinguicolla,Endangered,open,https://biodiversity.org.au/afd/taxa/5bceebc1-...,152228,73985,708886,,40,ACT Government,endangered,private,12986,"Australian Capital Territory, AU"


In [65]:
# prepare the export fields, common to New template and Update template
# new statuses
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username
mergedstatuses['new_authority'] = "Nature Conservation Act 2014 (ACT)"
mergedstatuses['new_description'] = "Listed as Threatened - refer to https://www.environment.act.gov.au/nature-conservation/conservation-and-ecological-communities/threatened-species-factsheets"
mergedstatuses['new_url'] = mergedstatuses['lsid']
mergedstatuses['new_geoprivacy'] = "obscured"
mergedstatuses['new_place_id'] = '12986'  # Australian Capital Territory
mergedstatuses['new_username'] = 'peggydnew'
mergedstatuses['new_iucn_equivalent'] = mergedstatuses['status'].str.lower().str.strip().map(iucnStatusMappings).fillna('Vulnerable') # map to dictionary
mergedstatuses['new_status'] = mergedstatuses['status'].fillna('Threatened')
mergedstatuses

,scientificName,status,bionet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,place_id,place_display_name,new_authority,new_description,new_url,new_geoprivacy,new_place_id,new_username,new_iucn_equivalent,new_status
43,Anthochaera phrygia,Critically Endangered,open,https://biodiversity.org.au/afd/taxa/31869a0e-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Nature Conservation Act 2014 (ACT),Listed as Threatened - refer to https://www.en...,https://biodiversity.org.au/afd/taxa/31869a0e-...,obscured,12986,peggydnew,Vulnerable,Critically Endangered
97,Aprasia parapulchella,Vulnerable,open,https://biodiversity.org.au/afd/taxa/0d74fa05-...,152221,36957,708886,,30,ACT Government,...,12986,"Australian Capital Territory, AU",Nature Conservation Act 2014 (ACT),Listed as Threatened - refer to https://www.en...,https://biodiversity.org.au/afd/taxa/0d74fa05-...,obscured,12986,peggydnew,Vulnerable,Vulnerable
98,Aprasia parapulchella,Vulnerable,open,https://biodiversity.org.au/afd/taxa/0d74fa05-...,152221,36957,708886,,30,ACT Government,...,12986,"Australian Capital Territory, AU",Nature Conservation Act 2014 (ACT),Listed as Threatened - refer to https://www.en...,https://biodiversity.org.au/afd/taxa/0d74fa05-...,obscured,12986,peggydnew,Vulnerable,Vulnerable
22,Aprasia parapulchella,Vulnerable,obscured,https://biodiversity.org.au/afd/taxa/0d74fa05-...,152221,36957,708886,,30,ACT Government,...,12986,"Australian Capital Territory, AU",Nature Conservation Act 2014 (ACT),Listed as Threatened - refer to https://www.en...,https://biodiversity.org.au/afd/taxa/0d74fa05-...,obscured,12986,peggydnew,Vulnerable,Vulnerable
21,Aprasia parapulchella,Vulnerable,obscured,https://biodiversity.org.au/afd/taxa/0d74fa05-...,152221,36957,708886,,30,ACT Government,...,12986,"Australian Capital Territory, AU",Nature Conservation Act 2014 (ACT),Listed as Threatened - refer to https://www.en...,https://biodiversity.org.au/afd/taxa/0d74fa05-...,obscured,12986,peggydnew,Vulnerable,Vulnerable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,Tympanocryptis pinguicolla,Endangered,obscured,https://biodiversity.org.au/afd/taxa/5bceebc1-...,152228,73985,708886,,40,ACT Government,...,12986,"Australian Capital Territory, AU",Nature Conservation Act 2014 (ACT),Listed as Threatened - refer to https://www.en...,https://biodiversity.org.au/afd/taxa/5bceebc1-...,obscured,12986,peggydnew,Vulnerable,Endangered
14,Tympanocryptis pinguicolla,Endangered,obscured,https://biodiversity.org.au/afd/taxa/5bceebc1-...,152228,73985,708886,,40,ACT Government,...,12986,"Australian Capital Territory, AU",Nature Conservation Act 2014 (ACT),Listed as Threatened - refer to https://www.en...,https://biodiversity.org.au/afd/taxa/5bceebc1-...,obscured,12986,peggydnew,Vulnerable,Endangered
63,Tympanocryptis pinguicolla,Endangered,open,https://biodiversity.org.au/afd/taxa/5bceebc1-...,152228,73985,708886,,40,ACT Government,...,12986,"Australian Capital Territory, AU",Nature Conservation Act 2014 (ACT),Listed as Threatened - refer to https://www.en...,https://biodiversity.org.au/afd/taxa/5bceebc1-...,obscured,12986,peggydnew,Vulnerable,Endangered
62,Tympanocryptis pinguicolla,Endangered,open,https://biodiversity.org.au/afd/taxa/5bceebc1-...,152228,73985,708886,,40,ACT Government,...,12986,"Australian Capital Territory, AU",Nature Conservation Act 2014 (ACT),Listed as Threatened - refer to https://www.en...,https://biodiversity.org.au/afd/taxa/5bceebc1-...,obscured,12986,peggydnew,Vulnerable,Endangered


## Updates

In [66]:
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
updates = pd.DataFrame(mergedstatuses[mergedstatuses['status_id'].notnull()])
updates.sort_values('scientificName')
updates['action'] = 'UPDATE'
#updates.loc[:,'action'] = 'UPDATE'
updates = updates[['action','scientificName','status_id','taxon_id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
updates.columns = updates.columns.str.replace("new_", "", regex=True)
updates = updates.rename(columns={'name':'taxon_name',
                                  'status_id':'id'})
updates

,action,scientificName,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
97,UPDATE,Aprasia parapulchella,152221,36957,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),https://biodiversity.org.au/afd/taxa/0d74fa05-...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
98,UPDATE,Aprasia parapulchella,152221,36957,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),https://biodiversity.org.au/afd/taxa/0d74fa05-...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
22,UPDATE,Aprasia parapulchella,152221,36957,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),https://biodiversity.org.au/afd/taxa/0d74fa05-...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
21,UPDATE,Aprasia parapulchella,152221,36957,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),https://biodiversity.org.au/afd/taxa/0d74fa05-...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
77,UPDATE,Bossiaea grayi,152220,795624,Endangered,Vulnerable,Nature Conservation Act 2014 (ACT),https://id.biodiversity.org.au/node/apni/2910201,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
76,UPDATE,Bossiaea grayi,152220,795624,Endangered,Vulnerable,Nature Conservation Act 2014 (ACT),https://id.biodiversity.org.au/node/apni/2910201,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
36,UPDATE,Bossiaea grayi,152220,795624,Endangered,Vulnerable,Nature Conservation Act 2014 (ACT),https://id.biodiversity.org.au/node/apni/2910201,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
35,UPDATE,Bossiaea grayi,152220,795624,Endangered,Vulnerable,Nature Conservation Act 2014 (ACT),https://id.biodiversity.org.au/node/apni/2910201,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
25,UPDATE,Cooraboorama canberrae,152231,781565,Rare,Vulnerable,Nature Conservation Act 2014 (ACT),https://biodiversity.org.au/afd/taxa/fa3dfc36-...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
83,UPDATE,Delma impar,152227,36937,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),https://biodiversity.org.au/afd/taxa/51c700dd-...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...


## No status in iNaturalist via straight scientificName match
The ACT records that didn't match up to a status in iNaturalist

In [67]:
# to add: those that have no inaturalist status - 532!!
noinatstatus = mergedstatuses[mergedstatuses['status_id'].isnull()]
# try to match the taxon name to something in inaturalist
noinatstatus = noinatstatus.merge(inattaxa, how="left", left_on="scientificName",right_on="scientificName")
noinatstatus

,scientificName,status,bionet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Anthochaera phrygia,Critically Endangered,open,https://biodiversity.org.au/afd/taxa/31869a0e-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Passeriformes,Meliphagidae,Anthochaera,phrygia,NaN,2022-06-09T16:08:32Z,species,http://www.birds.cornell.edu/clementschecklist...
1,Bettongia gaimardi,Conservation Dependent,open,https://biodiversity.org.au/afd/taxa/19c9bfdf-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Mammalia,Diprotodontia,Potoroidae,Bettongia,gaimardi,NaN,2020-07-20T02:39:44Z,species,http://www.catalogueoflife.org/annual-checklis...
2,Bidyanus bidyanus,Endangered,open,https://biodiversity.org.au/afd/taxa/05866f31-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Actinopterygii,Centrarchiformes,Terapontidae,Bidyanus,bidyanus,NaN,2019-11-24T03:53:09Z,species,http://www.fishbase.org
3,Botaurus poiciloptilus,Endangered,open,https://biodiversity.org.au/afd/taxa/dba78701-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Pelecaniformes,Ardeidae,Botaurus,poiciloptilus,NaN,2022-06-09T16:43:19Z,species,http://www.birdlife.org/datazone/speciesfactsh...
4,Caladenia actensis,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5139...,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Caladenia,actensis,NaN,2022-08-18T08:16:27Z,species,NaN
5,Caladenia actensis,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5139...,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Caladenia,actensis,NaN,2022-08-18T08:16:27Z,species,NaN
6,Caladenia actensis,Critically Endangered,open,https://id.biodiversity.org.au/taxon/apni/5139...,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Caladenia,actensis,NaN,2022-08-18T08:16:27Z,species,NaN
7,Caladenia actensis,Critically Endangered,open,https://id.biodiversity.org.au/taxon/apni/5139...,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Caladenia,actensis,NaN,2022-08-18T08:16:27Z,species,NaN
8,Caladenia montana,NaN,obscured,https://id.biodiversity.org.au/taxon/apni/5139...,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Caladenia,montana,NaN,2020-02-18T23:43:09Z,species,http://www.catalogueoflife.org/annual-checklis...
9,Calyptorhynchus lathami lathami,Vulnerable,open,https://biodiversity.org.au/afd/taxa/9823b764-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Psittaciformes,Cacatuidae,Calyptorhynchus,lathami,lathami,2018-12-19T06:38:49Z,subspecies,NaN


In [68]:
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions

,scientificName,status,bionet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Anthochaera phrygia,Critically Endangered,open,https://biodiversity.org.au/afd/taxa/31869a0e-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Passeriformes,Meliphagidae,Anthochaera,phrygia,NaN,2022-06-09T16:08:32Z,species,http://www.birds.cornell.edu/clementschecklist...
1,Bettongia gaimardi,Conservation Dependent,open,https://biodiversity.org.au/afd/taxa/19c9bfdf-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Mammalia,Diprotodontia,Potoroidae,Bettongia,gaimardi,NaN,2020-07-20T02:39:44Z,species,http://www.catalogueoflife.org/annual-checklis...
2,Bidyanus bidyanus,Endangered,open,https://biodiversity.org.au/afd/taxa/05866f31-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Actinopterygii,Centrarchiformes,Terapontidae,Bidyanus,bidyanus,NaN,2019-11-24T03:53:09Z,species,http://www.fishbase.org
3,Botaurus poiciloptilus,Endangered,open,https://biodiversity.org.au/afd/taxa/dba78701-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Pelecaniformes,Ardeidae,Botaurus,poiciloptilus,NaN,2022-06-09T16:43:19Z,species,http://www.birdlife.org/datazone/speciesfactsh...
4,Caladenia actensis,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5139...,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Caladenia,actensis,NaN,2022-08-18T08:16:27Z,species,NaN
5,Caladenia actensis,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5139...,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Caladenia,actensis,NaN,2022-08-18T08:16:27Z,species,NaN
6,Caladenia actensis,Critically Endangered,open,https://id.biodiversity.org.au/taxon/apni/5139...,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Caladenia,actensis,NaN,2022-08-18T08:16:27Z,species,NaN
7,Caladenia actensis,Critically Endangered,open,https://id.biodiversity.org.au/taxon/apni/5139...,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Caladenia,actensis,NaN,2022-08-18T08:16:27Z,species,NaN
8,Caladenia montana,NaN,obscured,https://id.biodiversity.org.au/taxon/apni/5139...,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Caladenia,montana,NaN,2020-02-18T23:43:09Z,species,http://www.catalogueoflife.org/annual-checklis...
9,Calyptorhynchus lathami lathami,Vulnerable,open,https://biodiversity.org.au/afd/taxa/9823b764-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Psittaciformes,Cacatuidae,Calyptorhynchus,lathami,lathami,2018-12-19T06:38:49Z,subspecies,NaN


In [69]:
# there's no status but there is a matching inat taxon (id is the taxon id)
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions.sort_values(['scientificName'])
additions['action'] = 'ADD'
additions = additions[['action','scientificName','status_id','id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
additions.columns = additions.columns.str.replace("new_", "", regex=True)
additions = additions.rename(columns={'scientificName':'taxon_name',
                                      'id':'taxon_id',
                                  'status_id':'id'})
additions

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
0,ADD,Anthochaera phrygia,NaN,144707,Critically Endangered,Vulnerable,Nature Conservation Act 2014 (ACT),https://biodiversity.org.au/afd/taxa/31869a0e-...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
1,ADD,Bettongia gaimardi,NaN,42996,Conservation Dependent,Vulnerable,Nature Conservation Act 2014 (ACT),https://biodiversity.org.au/afd/taxa/19c9bfdf-...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
2,ADD,Bidyanus bidyanus,NaN,95759,Endangered,Vulnerable,Nature Conservation Act 2014 (ACT),https://biodiversity.org.au/afd/taxa/05866f31-...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
3,ADD,Botaurus poiciloptilus,NaN,5032,Endangered,Vulnerable,Nature Conservation Act 2014 (ACT),https://biodiversity.org.au/afd/taxa/dba78701-...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
4,ADD,Caladenia actensis,NaN,1127611,Endangered,Vulnerable,Nature Conservation Act 2014 (ACT),https://id.biodiversity.org.au/taxon/apni/5139...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
5,ADD,Caladenia actensis,NaN,1127611,Endangered,Vulnerable,Nature Conservation Act 2014 (ACT),https://id.biodiversity.org.au/taxon/apni/5139...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
6,ADD,Caladenia actensis,NaN,1127611,Critically Endangered,Vulnerable,Nature Conservation Act 2014 (ACT),https://id.biodiversity.org.au/taxon/apni/5139...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
7,ADD,Caladenia actensis,NaN,1127611,Critically Endangered,Vulnerable,Nature Conservation Act 2014 (ACT),https://id.biodiversity.org.au/taxon/apni/5139...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
8,ADD,Caladenia montana,NaN,908175,Threatened,Vulnerable,Nature Conservation Act 2014 (ACT),https://id.biodiversity.org.au/taxon/apni/5139...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...
9,ADD,Calyptorhynchus lathami lathami,NaN,720267,Vulnerable,Vulnerable,Nature Conservation Act 2014 (ACT),https://biodiversity.org.au/afd/taxa/9823b764-...,obscured,12986,peggydnew,Listed as Threatened - refer to https://www.en...


In [70]:
# write these to the file
pd.concat([updates,additions]).to_csv(sourcedir + "act.csv", index=False)

In [71]:
# what didnt match to a taxon?
unknownToInat = noinatstatus[noinatstatus['id'].isna()]
unknownToInat

,scientificName,status,bionet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
11,Corunastylis ectopa,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Corunastylis ectopa,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Corunastylis ectopa,Critically Endangered,open,https://id.biodiversity.org.au/taxon/apni/5140...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Corunastylis ectopa,Critically Endangered,open,https://id.biodiversity.org.au/taxon/apni/5140...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Dasyurus maculatus maculatus,Vulnerable,open,https://biodiversity.org.au/afd/taxa/a4ef7496-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Gadopsis bispinosus,Vulnerable,open,https://biodiversity.org.au/afd/taxa/347bd80e-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Gentiana baeuerlenii,Endangered,open,https://id.biodiversity.org.au/node/apni/2892776,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,Gentiana baeuerlenii,Endangered,open,https://id.biodiversity.org.au/node/apni/2892776,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Gentiana baeuerlenii,Endangered,obscured,https://id.biodiversity.org.au/node/apni/2892776,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Gentiana baeuerlenii,Endangered,obscured,https://id.biodiversity.org.au/node/apni/2892776,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# inat statuses that aren't in added or updated
inatstatuses[~inatstatuses['taxon_id'].isin(updates['taxon_id'])]

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
1086,152232,568018,708886,12986,16649,ACT Government,rare,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Maratus,calcitrans,,2021-08-26T01:12:23Z,Maratus calcitrans,species,,,,


### are there any that need to be removed?
ACT sensitive list count: 25
ACT inat statuses count: 53

updates to inat status: 56
additional inat status: 45
ACT statuses we can't find a taxon match for in iNaturalist: 13
total: 78 (explainable via the various genus/section entries that we matched to in the taxonomy)

inat statuses left over:  that may need checking

In [73]:
# Stats
numsensitive = len(sensitivelist.index)
numconservation = len(conservationlist.index)
numupdates  = len(updates.index)
numadditions  = len(additions.index)
numnoinatstatus = len(noinatstatus.index)
numunknownToInat = len(unknownToInat.index)
numnoncomply = len(noncomply.index)
numcomply = len(statelist.index)
numdupinfo = len(dupinformation.index)
d = {'Sensitive': [numsensitive],
    'Conservation': [numconservation],
    'Statelist merge': [numfullstatelist],
    'Species iNat Comply' : [numcomply],
    'Species iNat non-Comply': [numnoncomply],
    'Duplicate Information': [numdupinfo],
    'Updates': [numupdates],
    'Additions': [numadditions],
    'No Inat Status': [numnoinatstatus],
    'Unknown to Inat': [numunknownToInat]}
statsdf = pd.DataFrame(data=d)
statsdf

,Sensitive,Conservation,Statelist merge,Species iNat Comply,Species iNat non-Comply,Duplicate Information,Updates,Additions,No Inat Status,Unknown to Inat
0,25,53,114,114,0,72,56,45,58,13


# inat statuses that aren't in added or updated
inatstatuses[~inatstatuses['taxon_id'].isin(updates['taxon_id'])]